## Краткий вывод о проделанной работе

### Подготовка данных
1. **Загрузка данных**: Данные были загружены из файлов `train_data.pqt` и `test_data.pqt`, содержащих соответственно 600,000 и 290,120 строк.
2. **Заполнение пропусков**: Были обработаны пропущенные значения в колонке `start_cluster` тестового набора данных методом заполнения предыдущими значениями.
3. **Определение кластеров**: Создан словарь `clusters` для предсказания конечных кластеров на основе начальных.
4. **Добавление признаков**: Добавлены новые столбцы `filled_cols` и `end_cluster_1` в тренировочный и тестовый наборы данных.

### Обучение модели
1. **Категориальные признаки**: Определены и преобразованы категориальные признаки в тип 'category'.
2. **Создание выборок**: Данные были разделены на тренировочную и валидационную выборки в соотношении 80/20.
3. **Обучение модели**: Обучена модель `LGBMClassifier` с заданными гиперпараметрами (learning_rate, max_depth, lambda_l2 и др.).
4. **Оценка модели**: Взвешенная метрика ROC AUC на валидационной выборке составила 0.943.

### Предсказание на тестовой выборке
1. **Обучение на всей выборке**: Модель была переобучена на всех тренировочных данных.
2. **Прогнозирование**: Предсказаны вероятности классов для тестовой выборки и сохранены в файл `lgbm_result.csv`.

### Подбор гиперпараметров
1. **Оптимизация гиперпараметров**: Проведен RandomizedSearchCV для поиска лучших гиперпараметров модели.
2. **Обучение с лучшими параметрами**: Модель с лучшими параметрами была обучена и проверена на валидационной выборке.

### Итоги
- **Качество модели**: Метрика ROC AUC на валидационной выборке показывает высокое качество модели.
- **Оптимизация**: Подбор гиперпараметров помог улучшить качество модели.

В результате была получена модель, которая эффективно прогнозирует вероятности конечных кластеров, что подтверждается высокими значениями метрики на валидационной выборке.

# Подготовка данных

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 10

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip freeze | grep "numpy\|pandas\|lightgbm\|scikit-learn\|seaborn\|matplotlib"

geopandas==0.13.2
lightgbm==4.1.0
matplotlib==3.7.1
matplotlib-inline==0.1.7
matplotlib-venn==0.11.10
numpy==1.25.2
pandas==2.0.3
pandas-datareader==0.10.0
pandas-gbq==0.19.2
pandas-stubs==2.0.3.230814
scikit-learn==1.2.2
seaborn==0.13.1
sklearn-pandas==2.2.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FILE_PATH = '/content/drive/MyDrive/ALFA_FinU/data/'

Mounted at /content/drive


In [ ]:
train_df = pd.read_parquet(FILE_PATH+'train_data.pqt')
test_df = pd.read_parquet(FILE_PATH+'test_data.pqt')
train_df.shape, test_df.shape

((600000, 93), (290120, 92))

In [ ]:
# Подсчет числа заполненных значений в каждой строке
train_df['filled_cols'] = train_df.drop(['id', 'date', 'start_cluster', 'end_cluster'], axis=1).count(axis=1)
test_df['filled_cols'] = test_df.drop(['id', 'date', 'start_cluster'], axis=1).count(axis=1)

Определение кластеров: Создается словарь clusters, который содержит соответствие начальных и конечных кластеров.

In [ ]:
# Наиболее вероятный конечный кластер для заданного начального кластера

clusters = {
 '{other}': '{other}',
 '{}': '{}',
 '{α, β}': '{α, β}',
 '{α, γ}': '{α, γ}',
 '{α, δ}': '{α}',
 '{α, ε, η}': '{α, ε, η}',
 '{α, ε, θ}': '{α, ε, θ}',
 '{α, ε, ψ}': '{α, ε, ψ}',
 '{α, ε}': '{α, ε}',
 '{α, η}': '{α, η}',
 '{α, θ}': '{α, θ}',
 '{α, λ}': '{α, λ}',
 '{α, μ}': '{α, μ}',
 '{α, π}': '{other}',
 '{α, ψ}': '{α, ψ}',
 '{α}': '{α}',
 '{λ}': '{α, λ}'
 }

Заполнение пропусков в кластерах: Пропущенные значения в столбце start_cluster тестового набора данных заполняются предыдущими значениями.

In [ ]:
# Заполнение начального кластера в month_6 значениями из month_5

test_df['start_cluster'].fillna(method='ffill', inplace=True)

Определение конечных кластеров: Добавляется новый столбец end_cluster_1 в обучающий и тестовый наборы данных, содержащий конечные кластеры для каждого начального кластера на основе словаря clusters

In [ ]:
# Предсказание конечного кластера по матрице вероятностей

train_df['end_cluster_1'] = train_df['start_cluster'].map(clusters)
test_df['end_cluster_1'] = test_df['start_cluster'].map(clusters)

# LGBM модель

Определение категориальных признаков: Создается список cat_cols, содержащий названия категориальных признаков.

In [ ]:
cat_cols = [
    "channel_code",
    "city",
    "city_type",
    "okved",
    "segment",
    "start_cluster",
    "index_city_code",
    "ogrn_month",
    "ogrn_year",
    'end_cluster_1'
]

Преобразование категориальных признаков в тип 'category'

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].astype("category")

Создаем выборки для валидации и обучения. Из обучающего набора данных удаляются столбцы "id", "date" и "end_cluster", которые не являются признаками для обучения модели.

In [ ]:
X = train_df.drop(["id", "date", "end_cluster"], axis=1)
y = train_df["end_cluster"]

Выбираются наиболее значимые признаки из списка top_features и создается выборка X, содержащая только эти признаки, а также вектор целевых значений y. Далее данные разделяются на обучающую и валидационную выборки в соотношении 80/20 с помощью функции train_test_split.


In [ ]:
# список фичей с наибольшей значимостью

top_features = ['filled_cols',
                'end_cluster_1',
    'start_cluster', 'okved', 'index_city_code', 'channel_code', 'city', 'balance_amt_min', 'segment', 'balance_amt_max', 'ogrn_days_end_quarter', 'sum_of_paym_1y', 'ogrn_days_end_month', 'cnt_a_oper_1m', 'min_founderpres',
 'ogrn_exist_months', 'ogrn_month', 'ft_registration_date', 'ogrn_year', 'sum_of_paym_6m', 'max_founderpres', 'sum_deb_e_oper_3m', 'balance_amt_avg', 'sum_of_paym_2m', 'sum_cred_e_oper_3m', 'cnt_days_deb_e_oper_3m',
 'balance_amt_day_avg', 'cnt_cred_e_oper_3m', 'sum_cred_e_oper_1m', 'sum_cred_h_oper_3m', 'cnt_days_cred_e_oper_3m',
 'cnt_deb_e_oper_3m', 'sum_deb_h_oper_3m', 'cnt_a_oper_3m', 'sum_deb_f_oper_3m', 'cnt_cred_e_oper_1m', 'sum_deb_g_oper_3m', 'sum_deb_e_oper_1m',
 'cnt_cred_h_oper_3m', 'cnt_deb_h_oper_3m', 'sum_deb_d_oper_3m', 'cnt_deb_e_oper_1m', 'sum_deb_f_oper_1m', 'sum_deb_h_oper_1m',
 'cnt_days_cred_e_oper_1m', 'cnt_deb_g_oper_3m', 'cnt_days_cred_h_oper_3m', 'sum_cred_g_oper_3m', 'sum_c_oper_3m',
 'sum_a_oper_3m', 'sum_cred_h_oper_1m', 'city_type', 'cnt_deb_d_oper_3m', 'sum_deb_d_oper_1m', 'cnt_days_deb_e_oper_1m', 'cnt_deb_f_oper_3m', 'sum_deb_g_oper_1m', 'cnt_days_deb_h_oper_3m', 'cnt_days_deb_f_oper_3m',
 'cnt_c_oper_3m', 'cnt_days_deb_g_oper_3m', 'cnt_b_oper_1m', 'cnt_days_deb_h_oper_1m', 'cnt_days_cred_h_oper_1m',
 'cnt_deb_f_oper_1m', 'sum_cred_f_oper_3m', 'cnt_days_cred_g_oper_1m', 'cnt_cred_g_oper_3m', 'sum_cred_d_oper_3m', 'sum_b_oper_3m', 'sum_c_oper_1m', 'sum_a_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_deb_h_oper_1m',
                'cnt_b_oper_3m', 'cnt_days_cred_f_oper_1m', 'cnt_c_oper_1m', 'cnt_days_deb_g_oper_1m',
                'cnt_days_deb_f_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_h_oper_1m', 'cnt_cred_d_oper_1m', 'sum_b_oper_1m', 'cnt_deb_d_oper_1m', 'cnt_cred_f_oper_3m',
 'cnt_cred_d_oper_3m', 'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'sum_cred_d_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_3m', 'cnt_days_cred_f_oper_3m']

selected_features = top_features[:53]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X[selected_features], y,test_size=0.2,random_state=42)

# Обучение модели LGBMClassifier

In [ ]:
model = LGBMClassifier(verbosity=-1, #вывод сообщений отключен
                       random_state=42, #  начальное значение для генератора псевдослучайных чисел
                       n_jobs=-1,# использование всех доступных ядер
                       learning_rate = 0.01,# cкорость обучения
                       max_depth=10, # максимальная глубина деревьев решений
                       lambda_l2 = 3,# Коэффициент регуляризации L2 (гребневая регуляризация)
                       n_estimators = 500) #  Количество деревьев решений
model.fit(x_train, y_train)

LGBMClassifier(lambda_l2=3, learning_rate=0.01, max_depth=10, n_estimators=500,
               n_jobs=-1, random_state=42, verbosity=-1)

**Функция для взвешенной метрики ROC AUC**

Эта функция вычисляет взвешенную метрику ROC AUC для многоклассовой задачи. Она принимает на вход истинные значения y_true, предсказанные вероятности y_pred, список меток классов labels и словарь весов weights_dict. Сначала функция вычисляет нормированные веса для каждого класса, затем вычисляет ROC AUC для каждого класса с помощью функции roc_auc_score, и, наконец, возвращает их взвешенную сумму.

In [ ]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

 cluster_weights.xlsx, содержит веса для каждого кластера. Веса преобразуются в словарь weights_dict, где ключами являются метки кластеров, а значениями - соответствующие веса.

In [ ]:
cluster_weights = pd.read_excel(FILE_PATH+'cluster_weights.xlsx').set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

Проверка работы модели на валидационной выборке. Модель используется для предсказания вероятностей классов на валидационной выборке с помощью метода predict_proba. Затем вычисляется метрика ROC AUC с учетом весов, которые были предварительно подготовлены.

In [ ]:
# Предсказание вероятностей для валидационной выборки
y_pred_proba = model.predict_proba(x_val)

# Вычисление взвешенной метрики ROC AUC
y_pred_proba.shape

(120000, 17)

In [ ]:
print("Метрика на валидации: ", weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict))

Метрика на валидации:  0.9430586606486161


# Прогноз на тестовой выборке

Обучаем модель model_full на всей обучающей выборке (X[selected_features]), используя признаки selected_features. Эта модель будет использоваться для предсказания на тестовой выборке.

In [ ]:
model_full = LGBMClassifier(verbosity=-1,
                            random_state=42,
                            n_jobs=-1,
                            learning_rate = 0.01,
                            max_depth=10,
                            lambda_l2 = 3,
                            n_estimators = 500)
model_full.fit(X[selected_features], y)

LGBMClassifier(lambda_l2=3, learning_rate=0.01, max_depth=10, n_estimators=500,
               n_jobs=-1, random_state=42, verbosity=-1)

sample_submission.csv содержит заголовок для файла предсказаний.

In [ ]:
sample_submission_df = pd.read_csv(FILE_PATH+"sample_submission.csv")

In [ ]:
sample_submission_df.shape

(100000, 18)

In [ ]:
sample_submission_df.head()

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,200001,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,200002,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,200003,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,200004,0.2,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


DataFrame last_m_test_df содержит только строки из тестового набора данных, соответствующие последнему месяцу.

In [ ]:
last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

Здесь модель используется для предсказания вероятностей классов для тестовой выборки на последнем месяце. Предсказанные вероятности записываются в DataFrame test_pred_proba_df.

In [ ]:
test_pred_proba = model_full.predict_proba(last_m_test_df[selected_features])
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

In [ ]:
test_pred_proba_df.shape

(100000, 17)

In [ ]:
test_pred_proba_df.head(10)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.010243,0.012793,0.024907,0.020239,0.002600,0.000083,0.000214,0.000100,0.002928,0.003960,0.028940,0.000105,0.001513,0.000003,0.000734,0.890616,0.000021
1,0.006444,0.643875,0.001265,0.003101,0.000500,0.000247,0.000362,0.000064,0.002201,0.009944,0.001835,0.000055,0.000522,0.000003,0.001594,0.327967,0.000022
2,0.775783,0.007613,0.009663,0.031277,0.015089,0.000901,0.003178,0.001160,0.030958,0.011500,0.020295,0.000302,0.002834,0.000010,0.018453,0.070929,0.000056
3,0.050799,0.604731,0.001781,0.003545,0.000341,0.000180,0.000225,0.000139,0.009089,0.016458,0.002420,0.000075,0.000280,0.000004,0.000199,0.309729,0.000006
4,0.050196,0.164757,0.003276,0.010358,0.002179,0.000520,0.000164,0.000033,0.002131,0.329940,0.005496,0.000086,0.001608,0.000006,0.000354,0.428886,0.000010
5,0.075562,0.012217,0.054331,0.087893,0.074003,0.000097,0.005741,0.000067,0.009162,0.003890,0.015108,0.011834,0.000492,0.000007,0.004046,0.645526,0.000025
6,0.005181,0.705339,0.001437,0.002984,0.000453,0.000078,0.000262,0.000027,0.000622,0.002986,0.000894,0.000376,0.000139,0.000023,0.001017,0.278170,0.000009
7,0.014664,0.012050,0.015630,0.030094,0.002750,0.000059,0.000124,0.000013,0.001257,0.022744,0.003853,0.001133,0.020998,0.000003,0.000487,0.874135,0.000005
8,0.057692,0.329282,0.002999,0.010563,0.001620,0.000828,0.000799,0.000045,0.004523,0.152124,0.005391,0.000107,0.000997,0.000007,0.000696,0.432296,0.000030
9,0.014289,0.235970,0.056080,0.092009,0.000798,0.000096,0.000263,0.000028,0.001815,0.015280,0.003791,0.000124,0.001060,0.000005,0.000421,0.577946,0.000025


Cоздается DataFrame с предсказанными вероятностями для тестовой выборки, и эти данные записываются в файл "lgbm_result.csv"

In [ ]:
sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("lgbm_result.csv", index=False)

# Подбор гиперпараметров

In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [5, 10, 15],
    'lambda_l2': [1, 2, 3, 5],
    'num_leaves': [31, 50, 70, 100],
    'min_child_samples': [20, 30, 40],
    'subsample': [0.8, 0.9, 1.0]
}

In [ ]:
# Создание объекта модели
model = LGBMClassifier(verbosity=-1, random_state=42, n_jobs=-1)

# Создание объекта RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=50, scoring='roc_auc', cv=3, verbose=2, random_state=42, n_jobs=-1)

# Обучение модели
random_search.fit(x_train, y_train)

# Просмотр лучших параметров и оценки качества модели
print("Best parameters found: ", random_search.best_params_)
print("Best ROC AUC score found: ", random_search.best_score_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
best_params = random_search.best_params_

model_best = LGBMClassifier(**best_params, verbosity=-1, random_state=42, n_jobs=-1)
model_best.fit(x_train, y_train)

# Проверка на валидационных данных
y_pred_proba = model_best.predict_proba(x_val)
print("Метрика на валидации: ", weighted_roc_auc(y_val, y_pred_proba, model_best.classes_, weights_dict))